In [1]:
# Import modules with shorthand
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import xml.etree.ElementTree as ET
#disable warnings
import warnings
warnings.filterwarnings("ignore")


from functions import convert_top, convert_bottomDV, convert_bottomI, structure   # for functions you need 
# to add (idx,data,tree,root,filename) when you call the function

from ClinicalProtocolDictionaries import TxSiteGroups, OrgansperTxSite, OrganColors, OrganCodes
# from ClinicalProtocolDictionaries import TxSiteGroups, OrgansperTxSite, OrganColors, OrganCodes



#***********Select the file that contains the dose constraints you want to use to create the clinical protocol************

filepath='//kodiak/physics/Dose Constraints/Dose Constraints - Eclipse Clinical Protocols Excel Sheets/'

csvfile='Thoracic - Lung EA5181 - 6000cGy30fx'

objectives = pd.read_csv(filepath + csvfile + '.csv')


#***************************Start of script********************************************************************
#Read info and create a data frame
csvfile=csvfile[:-2]

area, TxSite, tempdose=csvfile.split(' - ')
totaldose,fractions=tempdose.split('cGy')

doseprescription=totaldose.split('-')
doseprescription = [float(i) for i in doseprescription] 
numfractions=fractions.split('-')
numfractions = [int(i) for i in numfractions] 

finalfile = TxSite + ' ('  

fxdose=np.empty(len(doseprescription))

if len(numfractions) is 1:
      
    for i in range(0,len(doseprescription)-1,1):
        fxdose[i]=doseprescription[i]/numfractions[0]
        finalfile+=str(int(fxdose[i]))+'-'
        

    fxdose[len(doseprescription)-1]=doseprescription[len(doseprescription)-1]/numfractions[0]
    finalfile+= str(int(fxdose[len(doseprescription)-1]))+'cGy x ' +  str(numfractions[0]) + 'fx)'
        
elif len(numfractions)>1:

    fxdose[0]=doseprescription[0]/numfractions[0]
    finalfile+=str(int(fxdose[0]))+'cGy x ' +  str(numfractions[0]) + '-'
        
    for i in range(1,len(numfractions)-1,1):
        finalfile+= str(numfractions[i]) + '-'
    
        
    finalfile+= str(numfractions[len(numfractions)-1]) + 'fx)'  
    
finalfile
objectives

,Structure ID,Structure Code,Aliases,DVH Objective,Evaluator,Variation,Priority,Met,Achieved,Recommended vs considered,References (evaluator),References (Variation),Reviewed by,Comments / Suggestions
0,PTV_6000,NaN,NaN,V60Gy[%],>=95,Y,NaN,NaN,NaN,NaN,EA5181,NaN,NaN,NaN
1,PTV_6000,NaN,NaN,V54Gy[%],>=99,NaN,NaN,NaN,NaN,NaN,EA5181,NaN,NaN,D0.03cc <120%
2,PTV_6000,NaN,NaN,D0.03cc[Gy],<=72,75,NaN,NaN,NaN,NaN,EA5181,EA5181,NaN,NaN
3,PTV_6000,NaN,NaN,Min[Gy],>=57,NaN,NaN,NaN,NaN,NaN,EA5181,NaN,NaN,min>95%
4,ITV_6000,NaN,NaN,V60Gy[%],>=99.99,97,NaN,NaN,NaN,NaN,EA5181,EA5181,NaN,100% of ITV covered by the prescription dose
5,Lungs-CTV,NaN,NaN,V20Gy[%],<=35,37,NaN,NaN,NaN,NaN,EA5181,EA5181,NaN,NaN
6,Heart,NaN,NaN,V30Gy[%],<=40,45,NaN,NaN,NaN,NaN,EA5181,EA5181,NaN,NaN
7,Heart,NaN,NaN,V45Gy[%],<=30,35,NaN,NaN,NaN,NaN,EA5181,EA5181,NaN,NaN
8,SpinalCord,NaN,NaN,D0.03cc[Gy],<=48,50,NaN,NaN,NaN,NaN,EA5181,EA5181,NaN,NaN
9,Esophagus,NaN,NaN,Mean[Gy],<=34,NaN,NaN,NaN,NaN,NaN,EA5181,NaN,NaN,NaN


In [2]:

#___________________________________CONVERT DOSE CONSTRAINTS CSV FILE TO PLAN OBJECTIVES XML___________________________________

#*********** Create the dataframe with all the dose constraint info to pass on to the XML file ************

# XMLdata is that data frame that will contain all the dose constraint information necessary to convert the CSV dose constraints
# file into and XML

#Create the empty dataframe
XMLdata= pd.DataFrame(0,index=range(0,len(objectives)),columns=['ObjType','I1','I2','I3','I4','I5','I6'])

# Give it the structures that each objective refers to.  This comes from the "objectives" dataframe that reflects the info
# in the CSV datafile 
XMLdata.I1=objectives['Structure ID']

#figure out the indeces of the objectives that are "bottom" type in eclipse clinical protocol's plan objectives
Vobj= objectives['DVH Objective'].str.startswith('V')
Dobj=objectives['DVH Objective'].str.contains('D') 
GIobj= objectives['DVH Objective'].str.startswith('GI')
CIobj= objectives['DVH Objective'].str.startswith('CI')
bottom=Vobj|Dobj|GIobj|CIobj

#Tag each objective with the "top" or "bottom" classification so they can be properly formated in the XML file
XMLdata.ObjType[bottom]='bottom'
XMLdata.ObjType[~bottom]='top'


#Translate each objective into its "Type" value according to the 'dictionary for plan objectives', and set the XMLdata 
# value accordingly
#do not have to worry about setting the "0" types because 0 is the default value of the XMLdata matrix
# Type0=CIobj
Type1=GIobj
Type2=objectives['DVH Objective'].str.startswith('V') & ~objectives['DVH Objective'].str.contains('Gy')
Type3=objectives['DVH Objective'].str.startswith('V') & objectives['DVH Objective'].str.contains('Gy')
Type4=objectives['DVH Objective'].str.startswith('D') & ~objectives['DVH Objective'].str.contains('cc')
Type5=objectives['DVH Objective'].str.startswith('D') & objectives['DVH Objective'].str.contains('cc')


XMLdata.I2[Type1]=1
XMLdata.I2[Type2]=2
XMLdata.I2[Type3]=3
XMLdata.I2[Type4]=4
XMLdata.I2[Type5]=5

# Translate evaluators into modifier values.  Again, for more specifics on what each modifier value means, refer to the 
# 'dictionary for plan objectives' file
#the default XMLdata value is 0, so we don't have to worry about setting 0 values.
# mod5 and mod6 are not defined because mod5 is never used in our clinic, and mod6 has no definition we could find based on
# the approach we took to develop the dictionary for plan objectives

Mod1top=objectives['Evaluator'].str.startswith('<') & XMLdata.ObjType.isin(['top']) 
Mod2top=objectives['DVH Objective'].str.contains('mean', case=False) & objectives['Evaluator'].str.startswith('=') & XMLdata.ObjType.isin(['top']) 
Mod3top=objectives['DVH Objective'].str.contains('max', case=False) & objectives['Evaluator'].str.startswith('=')  & XMLdata.ObjType.isin(['top'])
Mod4top=objectives['DVH Objective'].str.contains('min', case=False) & objectives['Evaluator'].str.startswith('=')  & XMLdata.ObjType.isin(['top']) 
#Mod5top=objectives['DVH Objective'].str.contains('max', case=False) & objectives['Evaluator'].str.startswith('=')  & XMLdata.ObjType.isin(['top']) 
#Mod6top we didn't find any objectives with a modifier value of 6 when creating our dictionary
Mod7top=objectives['DVH Objective'].str.contains('mean', case=False) & objectives['Evaluator'].str.startswith('>')  & XMLdata.ObjType.isin(['top']) 
Mod8top=objectives['DVH Objective'].str.contains('mean', case=False) & objectives['Evaluator'].str.startswith('<')  & XMLdata.ObjType.isin(['top']) 
Mod9top=objectives['DVH Objective'].str.contains('min', case=False) & objectives['Evaluator'].str.startswith('>')  & XMLdata.ObjType.isin(['top']) 
Mod10top=objectives['DVH Objective'].str.contains('max', case=False) & objectives['Evaluator'].str.startswith('<')  & XMLdata.ObjType.isin(['top']) 

Mod0bot=objectives['Evaluator'].str.startswith('>') & XMLdata.ObjType.isin(['bottom']) 
Mod1bot=objectives['Evaluator'].str.startswith('<') & XMLdata.ObjType.isin(['bottom']) 
Mod2bot=objectives['Evaluator'].str.startswith('=') & XMLdata.ObjType.isin(['bottom']) 

XMLdata.I3[Mod1top]=1
XMLdata.I3[Mod2top]=2
XMLdata.I3[Mod3top]=3
XMLdata.I3[Mod4top]=4
#XMLdata.I3[Mod5top]=5
#XMLdata.I3[Mod6top]=6
XMLdata.I3[Mod7top]=7
XMLdata.I3[Mod8top]=8
XMLdata.I3[Mod9top]=9
XMLdata.I3[Mod10top]=10

# XMLdata.I3[Mod0bot]=0
XMLdata.I3[Mod1bot]=1
XMLdata.I3[Mod2bot]=2

# Set the parameter values for the "top".  
# Only need to set the parameter for mod 4 and 9, all the other ones are 0
XMLdata.I4[objectives['DVH Objective'].str.contains('min', case=False)]=100
#parameter for mod 0 and 1 - do we have this capability in our current excel sheet template?

# Set the "value" aka target values for the "bottom".  
for x in range(0,len(objectives),1):
    
    if bottom[x]:
        
        #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
        s=objectives.iloc[x,4]  
        
        #take out the [Gy],[cc],[%]
        stest=re.split('(\d+\D*\d*)',s)  #this notation means 1 or more digits followed by 0 or more special characters, followed by 0 or more digits
        
        if objectives.iloc[x,3].endswith('[cc]'):
        #split the first portion so you can extract the number
            XMLdata.I4[x]=str(float(stest[1])*1000) #convert cc to mm3
        else:
            XMLdata.I4[x]=str(float(stest[1]))
        
# Set the type specifier for the "bottom" and whether or not the units should be absolute or relative
for x in range(0,len(objectives),1):
    
    if Vobj[x]|Dobj[x]:
        
        #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
        t=objectives.iloc[x,3]  
        
        #take out the [Gy],[cc],[%]
        ttest=re.split('(\D+\[\D*\])',t)  #this notation means 1 or more non-numbers followed by [, followed by 0 or more non-numbers, followed by ]
        
        #split the first portion so you can extract the number
        ta=re.split('(\d+\D*\d*)',ttest[0]) #this means one or more digits followed by 0 or more non-numbers, followed by zero or more digits
        
        #convert the digit to a float to set the typespecifier for type 2,3,4,5
        XMLdata.I5[x]=str(ta[1]) 
        
        #check if the DQPvalue is in absolute units or not
        if re.search('\[%\]',ttest[1])==None:  
            XMLdata.I6[x]="true"
        else:
            XMLdata.I6[x]="false"
     
        
        if Type5[x]:
            XMLdata.I5[x]=str(float(XMLdata.I5[x]))  
    
    elif GIobj[x]|CIobj[x]:
        XMLdata.I5[x]="true"
        XMLdata.I6[x]="false"
        
#Set the dose (dose per fraction) and total dose for the "top" objectives

for x in range(0,len(objectives),1):
    
    if not bottom[x]:
        
        #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
        s=objectives.iloc[x,4]  
        
        #take out the [Gy],[cc],[%]
        stest=re.split('(\d+\D*\d*)',s)  #this notation means 1 or more digits followed by 0 or more special characters, followed by 0 or more digits
        
        #split the first portion so you can extract the number
        #calculate the dose per fraction
        XMLdata.I5[x]=str(float(stest[1])/numfractions[0]) #convert the digit to a float and divide it by number of fractions assumes Gy in clinical protocol
        
        #set the total dose value
        XMLdata.I6[x]=str(float(stest[1])) #convert the digit to a float set it as the TotalDose
    
    
av = 0  #index to keep track of how many acceptable variations there have been 

for x in range(0,len(objectives),1):

    if not pd.isnull(objectives.Variation[x]):
        
        print(str(x) + ' is not null:' + str(objectives.Variation[x])) #this indicates there is a variation for that objective
        

        if (type(objectives.Variation[x]) is not str) | str.isdigit(str(objectives.Variation[x])):# | objectives.Variation[x].replace('.','0').isdigit():        
            
            #this copies the data of the objective for which there is a variation so it can be inserted as a new objective in 
#             the XML file
            variation = pd.DataFrame(XMLdata.loc[x+av,:]).transpose()
            
            
        
            if bottom[x]:
                
                #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
                s=objectives.Variation[x] 
                
                variation.I4=str(s)
                
                print ('bottom variation for ' + str(x) + ':' + str (variation))
                
                XMLdata = pd.concat([XMLdata.iloc[:x+1+av], variation, XMLdata.iloc[(x+1+av):]], ignore_index=True)
                
                av = av + 1
        
            else:
                
                #extract the DHV Objective column value for row x belonging to Type2, 3, 4, or 5
                s=objectives.Variation[x]  

                #split the first portion so you can extract the number
                variation.I5=str(float(s)/numfractions[0]) #convert the digit to a float and divide it by number of fractions assumes Gy in clinical protocol
                variation.I6=str(float(s)) #convert the digit to a float set it as the TotalDose  
                
                print ('top variation for ' + str(x) + ':' + str (variation))
            
                XMLdata = pd.concat([XMLdata.iloc[:x+1+av], variation, XMLdata.iloc[(x+1+av):]], ignore_index=True)
                av = av + 1
        
        else: print('variation in next line!')
    else: print('no acceptable variation')
        
#CREATE THE PLAN OBJECTIVES XLM DATA FEED.  Conserve the order of the dose constraints
XML_PO=XMLdata[XMLdata.ObjType.isin(['top'])].reset_index(drop=True) 
XML_PO=XML_PO.append(XMLdata[XMLdata.ObjType.isin(['bottom'])]).reset_index(drop=True)        

#******************* Write out the data in XML format ********************************

#use the XML file below as the template to create all clinical protocols:
ClinicalProtocol = ET.parse('Clinical Protocol Template.xml')
tree=ClinicalProtocol
Protocol=ClinicalProtocol.getroot()
root=Protocol


# this just adds the correct extension to the file name for the clinical protocol
filename = finalfile +'.xml'


#these are the indexes that will ensure the objectives are written correctly on the XML file
idxt = 1  #index for top objectives
idxb = 1 #index for bottom Dose/Volume objectives


#insert the objectives following the correct format for each type

for x in range(0,len(XML_PO),1):
    data=XML_PO.loc[x,'I1':'I6'] #do it one objective at a time (one row of the dataframe per objective)
    
    if XML_PO.ObjType[x]=='top':
        convert_top(idxt,data,ClinicalProtocol,Protocol,filename) 
        idxt = idxt +1

        
    elif XML_PO.I2[x]>=2:  # all objectives of type 2 or larger are "bottom dose/volume" objectives
        convert_bottomDV(idxb,data,ClinicalProtocol,Protocol,filename) 
        idxb = idxb+1
        
    else:
        convert_bottomI(idxb,data,ClinicalProtocol,Protocol,filename)   
        idxb = idxb+1
        
# set the number of fractions 

root.find('.//Phase//FractionCount').text = str(numfractions[0])  

# Change the clinical protocol name to the final file name so it shows the correct name when it's imported into Eclipse:
Protocol.find('.//Preview').set('ID', finalfile)
ClinicalProtocol.write(filename)


# ___________________________ END OF THE PLAN OBJECTIVES CSV TO XML CONVERSION_______________________________________
                


0 is not null:Y
variation in next line!
no acceptable variation
2 is not null:75
bottom variation for 2:  ObjType        I1 I2 I3  I4    I5    I6
2  bottom  PTV_6000  5  1  75  0.03  true
no acceptable variation
4 is not null:97
bottom variation for 4:  ObjType        I1 I2 I3  I4  I5     I6
5  bottom  ITV_6000  3  0  97  60  false
5 is not null:37
bottom variation for 5:  ObjType         I1 I2 I3  I4  I5     I6
7  bottom  Lungs-CTV  3  1  37  20  false
6 is not null:45
bottom variation for 6:  ObjType     I1 I2 I3  I4  I5     I6
9  bottom  Heart  3  1  45  30  false
7 is not null:35
bottom variation for 7:   ObjType     I1 I2 I3  I4  I5     I6
11  bottom  Heart  3  1  35  45  false
8 is not null:50
bottom variation for 8:   ObjType          I1 I2 I3  I4    I5    I6
13  bottom  SpinalCord  5  1  50  0.03  true
no acceptable variation
10 is not null:70
bottom variation for 10:   ObjType              I1 I2 I3  I4   I5    I6
16  bottom  BrachialPlex_L  5  1  70  0.5  true
11 is not null:7

In [3]:
#__________________________________ CREATE STRUCTURE TEMPLATE FOR CLINICAL PROTOCOL ___________________________________

#data info that will be used to create the XML structures:  
#
# 0 - structure name, 1 - type of structure, 2 - TypeIndex (always 2 for now), 3 - ColorAndStyle, 4 - DVHLineStyle (always 0 for now), 
# 5 - DVHLineColor (always -16777216), 6 - DVHLineWidth (always 1 for now)

#Create the XML matrix for the structure data
#Create the structure set based on the standardized list of organs suggested by PRO 2019 ASTRO consensus paper
# StructuresOfInterest=OrgansperTxSite[TxSite]

# or create the structure set based on the dose constraint sheet
StructuresOfInteresttemp=objectives['Structure ID'].unique()


# add any additional structures beyond what the dose constraint sheet lists

if 'Lung' in TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['Carina',
          'Liver',
          'Stomach',
          'Trachea']
         )
    print('Lung')

elif ('CW' in TxSite) or ('Breast' in TxSite) :
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Larynx',
              'Liver',
              'Stomach',
              'Esophagus',
              'SpinalCord',
              'Thyroid']
             )  
    print('CW ' + ' Breast')
    
    
    if ('Breast L' in TxSite) :
        StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
                  ['Lumpectomy_L']
                 )  
        print('Breast L')
    
    if ('Breast R' in TxSite) :
        StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
                  ['Lumpectomy_R']
                 )  
        print('Breast R')
        
    if ('R + n A221505' in TxSite) :
        StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
                  ['Scar_R']
                 )  
        print('A221505 R')   
        
    if ('L + n A221505' in TxSite) :
        StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
                  ['Scar_L']
                 )  
        print('A221505 L')   
        

elif 'Liver' in TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Liver^Ex',
              'Liver^In',
              'Lungs']
             ) 
    print('Liver')
    
elif 'Pancreas' in TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Kidney_L',
              'Kidney_R',
              'Lungs']
             ) 
    print('Pancreas')
    
elif 'Esophagus' in TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Kidney_L',
              'Kidney_R',
              'Esophagus']
             ) 
    print('Esophagus')
    
elif ('Endometrium'  in TxSite) or ('Anus'  in TxSite) or ('Cervix' in TxSite) or ('Vulva or Vagina' in TxSite) :
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['BoneMarrow']
             ) 
    print('Endometrium/Anus/Cervix/Vulva/Vagina' )
    

elif 'Brain' in TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Brain',
#               'Glnd_Lacrimal',
#               'Hippocampus_L',
#               'Hippocampus_R',
#               'Pituitary',
#               'Scalp',
#               'A_Carotid',
              'OpticChiasm',
              'OpticNrv_L',
              'OpticNrv_R',
              'Lens_L',
              'Lens_R',
              'Eye_L',
              'Eye_R',
              'Cochlea_L',
              'Cochlea_R',
              'SpinalCord']             
             ) 
    print('Brain') 
    
#     if 'ACNS1721' in TxSite :
#         StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
#                   ['GTV_Pre-Op',
#                   'GTV_Cavity']
#                  )  
#         print('ACNS1721')  

elif 'Abdomen'  in TxSite:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
              ['Lungs']
             ) 
    print('Abdomen' )
    
    
# elif 'Cervix' in TxSite:
#     StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
#               ['BoneMarrow']
#              ) 
#     print('Endometrium/Anus/Cervix/Vulva/Vagina' )
    
# elif 'Vulva or Vagina' in TxSite:
#     StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
#               ['BoneMarrow']
#              ) 
#     print('Endometrium/Anus/Cervix/Vulva/Vagina' )
   
    
# or create the structure set based on the dose constraint sheet
StructuresOfInterest=np.unique(StructuresOfInteresttemp)




Lung


In [4]:


# Create a dataframe to capture all the relevant information for the structures listed for that treatment site
XML_S= pd.DataFrame(0,index=range(0,len(StructuresOfInterest)),columns=
                    ['StrID','Type','TypeIdx','Color','DVHLS','DVHLC','DVHLW','iCode','iCodeScheme','iCodeSchemeVersion'])

# Set the structure names based on the list of organs for that treatment site
XML_S.StrID=StructuresOfInterest 

#  Set all the default values
XML_S.TypeIdx=2   
XML_S.DVHLS=0    #Style
XML_S.DVHLC= -16777216  #Color
XML_S.DVHLW=1    #Width
XML_S.iCodeScheme='FMA'
XML_S.iCodeSchemeVersion='3.2'

# find the colors that correspond to each organ using the OrganColors dictionary.  Organs that are derived like Lungs-ITV or 
# _PRV will have the same color as their original organs


XML_Stemp=XML_S 

        
av=0

for x in range (0,len(XML_Stemp),1):
    
    if XML_Stemp.StrID[x] in OrganColors: # if the structure is in the organ list, simply assign it the corresponding color
        XML_S.Color[x+av]=OrganColors[XML_Stemp.StrID[x]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[XML_Stemp.StrID[x]]
        
        
    elif '-' in XML_Stemp.StrID[x]: # if the structure is a derived structure from one found in the organ list,
                                #simply assign it the corresponding color
        temp=XML_Stemp.StrID[x].split('-') 
        
        XML_S.Color[x+av]=OrganColors[temp[0]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[temp[0]]
        
        
        
        trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
        trueStr.StrID=temp[0]
        trueStr.Color=OrganColors[temp[0]]
        trueStr.Type='Organ'        
        trueStr.iCode=OrganCodes[temp[0]]
        
        XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
        av = av + 1
        
        
    elif '_PRV' in XML_Stemp.StrID[x]: # if the structure is a derived structure split it first,                                 
        
        temp=XML_Stemp.StrID[x].split('_PRV') 
        print(temp)
    
        if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color             
            XML_S.Color[x+av]=OrganColors[temp[0]]
            XML_S.Type[x+av]='Organ'
            XML_S.iCode[x+av]='PRV'
            
            trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
            
            trueStr.StrID=temp[0]
            trueStr.Color=OrganColors[temp[0]]
            trueStr.Type='Organ'        
            trueStr.iCode=OrganCodes[temp[0]] 
        
            XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
            av = av + 1
            
    elif 'TV' in XML_Stemp.StrID[x]:
        XML_S.Type[x+av]='Target'

    elif ('E-PTV' in XML_Stemp.StrID[x]):
        XML_S.Type[x+av]='Organ'
        
    else:
        print (StructuresOfInterest[x] + ' not found')
        
# Drop the duplicate structures      
if len(XML_S.StrID) != len(set(XML_S.StrID)):
    XML_S=XML_S.drop_duplicates(subset='StrID')  
    XML_S=XML_S.reset_index(drop=True) 

    
TargetColors=['Segment - Red','Segment - Orange','Segment - Pink', 'RGB255200150'] #set standard PTV colors

targets=XML_S[XML_S.Type.isin(['Target'])]  #find the structures set as "target" in the structure set from the dose constraints

ignorePorC=targets  #fill in a temporary array


for x in range (0,len(targets),1):
    ignorePorC.StrID[targets.index[x]]=targets.StrID[targets.index[x]][1:]  #take out the first letter of the target structure to give a given PTV, CTV, GTV
                                              #the same color
    
TargetVolumes=ignorePorC.StrID.unique()      #figure out how many unique targets we have

if len(doseprescription)>1:
    TargetVolumes=sorted(TargetVolumes,reverse=False)
    
for x in range (0,len(TargetVolumes),1):
    XML_S.Color[XML_S['StrID'].str.contains(TargetVolumes[x])]=TargetColors[x]     #assigning them the specific colors
    
avt=0

#create GTV or CTV if there are none.

for x in range (0,len(TargetVolumes),1):
    
    avt=0

        
    if ('G'+TargetVolumes[x] not in XML_S['StrID']) and ('CW' not in TxSite) and ('Breast' not in TxSite) :    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)
        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()

        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='G' + temp[0]
        else:
            TVdef.StrID='G' + TargetVolumes[x]
      
        

        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        avt=1+avt

    if 'C'+TargetVolumes[x] not in XML_S['StrID']:    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='C' + temp[0]

        else:
            TVdef.StrID='C' + TargetVolumes[x]            
        

        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        avt=1+avt
        
        
    if TargetVolumes[x].endswith('_Eval'):    #create a PTV volume without the _Eval
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        temp=TargetVolumes[x].split('_Eval')
        TVdef.StrID='P' + temp[0]   
        

        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        avt=1+avt
        
        

XML_S.Type[XML_S.StrID.str.startswith('PTV')]='PTV'
XML_S.iCode[XML_S.StrID.str.startswith('PTV')]='PTVp'


XML_S.Type[XML_S.StrID.str.startswith('ITV')]='CTV' #no ITV available at the moment
XML_S.iCode[XML_S.StrID.str.startswith('ITV')]='ITV'


XML_S.Type[XML_S.StrID.str.startswith('CTV')]='CTV'
XML_S.iCode[XML_S.StrID.str.startswith('CTV')]='CTVp'


XML_S.Type[XML_S.StrID.str.startswith('GTV')]='GTV'
XML_S.iCode[XML_S.StrID.str.startswith('GTV')]='GTVp'
XML_S.iCode[XML_S.StrID.str.startswith('GTVn')]='GTVn'

for x in range(0,len(XML_S),1):
    
    if not str.isdigit(str(XML_S.iCode [x])):
        XML_S.iCodeScheme[x]='99VMS_STRUCTCODE'
        XML_S.iCodeSchemeVersion[x]='1.0'

XML_S=XML_S.drop_duplicates(subset='StrID').reset_index(drop=True)  


In [5]:
# This will create the structures in the XML file of the clinical protocol
idxs = 1

for x in range(0,len(XML_S),1):
    
    data=XML_S.loc[x,'StrID':'iCodeSchemeVersion'] #One structure per row of the dataframe
    
    if XML_S.Type[x] != 0: 
        
        structure(idxs,data,tree,root,filename)
        idxs= idxs + 1
        print('success!!! ' + str(x) + ':' + data[0] )  
        
    else: 
        print ('Skipped: ' + XML_S.StrID[x])  
         
        
        
# Save the clinical protocol XML file in a specific folder
folderpath='//kodiak/physics/Dose Constraints/Scripts/Completed Clinical Protocols for Eclipse/'
ClinicalProtocol.write(folderpath + filename)

os.remove('//kodiak/physics/Dose Constraints/Scripts/' + filename)
    
    

success!!! 0:BrachialPlex_L
success!!! 1:BrachialPlex_R
success!!! 2:Carina
success!!! 3:Esophagus
success!!! 4:Heart
success!!! 5:ITV_6000
success!!! 6:Liver
success!!! 7:Lungs
success!!! 8:Lungs-CTV
success!!! 9:GTV_6000
success!!! 10:PTV_6000
success!!! 11:CTV_6000
success!!! 12:SpinalCord
success!!! 13:Stomach
success!!! 14:Trachea


In [ ]:
#*************************************************************************************************************************
# This creates the blank clinical protocol template.  YOU DO NOT NEED TO RUN THIS UNLESS SOMETHING HAPPENS TO THE 
# TEMPLATE AND IT NEEDS TO BE REDONE

# ClinicalProtocol = ET.parse('Clinical protocol test.xml')
# tree=ClinicalProtocol
# Protocol=ClinicalProtocol.getroot()
# root=Protocol
# filename='Clinical Protocol Template.xml'

# for child in Protocol.find('.//Prescription'): 
#     Protocol.find('.//Prescription').remove(child)
#     ClinicalProtocol.write(filename)

# Protocol.find('.//Structures').clear()
# Protocol.find('.//Phases//Phase//FractionCount').clear()
# root.find('.//Phases//Phase//FractionCount').text ='25'
# Protocol.find('.//Phases//PlanTemplate//FractionCount').clear()
# root.find('.//Phases//PlanTemplate//FractionCount').text ='25'

# Protocol.find('.//Preview').set('ID', 'Clinical Protocol Template')
# ClinicalProtocol.write(filename)
